**Installing necessary packages and importing necessary libraries**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!pip install -q accelerate torch simpletransformers datasets evaluate
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import datasets
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, DataCollatorWithPadding, Trainer, TrainingArguments, AutoModelForSequenceClassification, pipeline
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix, classification_report, f1_score, precision_score, recall_score

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.8/250.8 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 59.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 71.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

**Reading the dataset**

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/sentiment.csv')
df.head(3)

,Sentiment,content
0,0,don't waste your money broken plates
1,0,waste of money! the phone keep falling as ther...
2,0,not specified not happy


**Checking class distribution**

In [ ]:
df['Sentiment'].value_counts()

0    4939
1    4404
Name: Sentiment, dtype: int64

**Dataset is balanced**

**Checking for and removing duplicates**

In [ ]:
print (df.shape)
df = df.drop_duplicates()
print (df.shape)

(9343, 2)
(9343, 2)


**No duplicates found**

**Checking for missing values**

In [ ]:
df.isnull().sum()

Sentiment    0
content      0
dtype: int64

**No missing values found**

**Renaming target variable**

In [ ]:
df.rename(columns = {'Sentiment':'sentiment'}, inplace = True)
df.head(1)

,sentiment,content
0,0,don't waste your money broken plates


**Making train, test and validation splits**

In [ ]:
y_to_stratify_df = df['sentiment']
train_old, test = train_test_split (df, test_size=0.2, random_state=0, stratify = y_to_stratify_df, shuffle = True)

y_to_stratify_train = train_old['sentiment']
train, val = train_test_split (train_old, test_size=0.2, random_state=0, stratify = y_to_stratify_train, shuffle = True)

**Converting the dataset to arrows format for faster processing**

In [ ]:
train = Dataset.from_pandas(train)
test = Dataset.from_pandas(test)
val = Dataset.from_pandas(val)

dataset = DatasetDict()

dataset['train'] = train
dataset['test'] = test
dataset['val'] = val
dataset

DatasetDict({
    train: Dataset({
        features: ['sentiment', 'content', '__index_level_0__'],
        num_rows: 5979
    })
    test: Dataset({
        features: ['sentiment', 'content', '__index_level_0__'],
        num_rows: 1869
    })
    val: Dataset({
        features: ['sentiment', 'content', '__index_level_0__'],
        num_rows: 1495
    })
})

**Removing index column**

In [ ]:
dataset = dataset.remove_columns ('__index_level_0__')
train = train.remove_columns ('__index_level_0__')
test = test.remove_columns ('__index_level_0__')
val = val.remove_columns ('__index_level_0__')
dataset

DatasetDict({
    train: Dataset({
        features: ['sentiment', 'content'],
        num_rows: 5979
    })
    test: Dataset({
        features: ['sentiment', 'content'],
        num_rows: 1869
    })
    val: Dataset({
        features: ['sentiment', 'content'],
        num_rows: 1495
    })
})

**Intitializing bert tokenizer**

In [ ]:
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

**Creating the function for tokenization**

In [ ]:
def tokenize_function(example):
    return tokenizer(example["content"], truncation=True)

**Tokenizing the dataset**

In [ ]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets

Map:   0%|          | 0/5979 [00:00<?, ? examples/s]

Map:   0%|          | 0/1869 [00:00<?, ? examples/s]

Map:   0%|          | 0/1495 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentiment', 'content', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 5979
    })
    test: Dataset({
        features: ['sentiment', 'content', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1869
    })
    val: Dataset({
        features: ['sentiment', 'content', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1495
    })
})

**Removing unncessary columns from tokenized datasets, renaming target variable and converting datasets to torch format**

In [ ]:
tokenized_datasets = tokenized_datasets.remove_columns(["content"])
tokenized_datasets = tokenized_datasets.rename_column("sentiment", "labels")
tokenized_datasets.set_format("torch")
tokenized_datasets["train"].column_names

['labels', 'input_ids', 'token_type_ids', 'attention_mask']

**Initializing data collator and bert model**

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


**Defining a custom fuction for calculating classification metrics with each epoch**

In [ ]:
def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted', zero_division=1)
  acc = accuracy_score(labels, preds)
  return {'accuracy': acc, 'f1': f1, 'precision': precision, 'recall': recall}

**Setting/Defining training arguments and incorporating them in the trainer**

In [ ]:
training_args = TrainingArguments("test-trainer", evaluation_strategy="epoch", num_train_epochs=10,
                                  per_device_train_batch_size=16, per_device_eval_batch_size=64, warmup_steps=500,
                                  weight_decay=0.01, logging_dir='./logs', logging_steps=10)

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

trainer = Trainer(model, training_args, train_dataset=tokenized_datasets["train"],
                  eval_dataset=tokenized_datasets["val"], data_collator=data_collator, tokenizer=tokenizer,
                  compute_metrics=compute_metrics)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


**Model training**

In [ ]:
trainer.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.264000,0.183429,0.941806,0.941550,0.944923,0.941806
2,0.042300,0.188371,0.957191,0.957191,0.957191,0.957191
3,0.052800,0.195294,0.954515,0.954467,0.954835,0.954515
4,0.018300,0.223014,0.957191,0.957124,0.957835,0.957191
5,0.047400,0.267117,0.960535,0.960524,0.960555,0.960535
6,0.030500,0.254344,0.958528,0.958518,0.958539,0.958528
7,0.001600,0.285092,0.958528,0.958528,0.958528,0.958528
8,0.006500,0.298416,0.958528,0.958485,0.958859,0.958528
9,0.000100,0.306593,0.958528,0.958494,0.958739,0.958528
10,0.006100,0.315385,0.958528,0.958511,0.958579,0.958528


TrainOutput(global_step=3740, training_loss=0.05936979860233742, metrics={'train_runtime': 890.0458, 'train_samples_per_second': 67.176, 'train_steps_per_second': 4.202, 'total_flos': 1953585711050940.0, 'train_loss': 0.05936979860233742, 'epoch': 10.0})

**Making predictions on test dataset**

In [ ]:
predictions = trainer.predict(tokenized_datasets["test"])

**Storing the target variable of test set in a variable named 'y_test' and converting it to a numpy array**

In [ ]:
y_test = tokenized_datasets['test']['labels']
y_test = np.array (y_test)

**Storing predictions in a variable named 'y_pred'**

In [ ]:
y_pred = np.argmax(predictions.predictions, axis=-1)

**Model evaluation**

In [ ]:
print ("Accuracy on test set:", accuracy_score (y_pred, y_test))
print ("Correct predictions on test set:", sum (y_test == y_pred))
print ("Incorrect predictions on test set:", sum (y_test != y_pred))
print ("F1 score on test set:", f1_score(y_test, y_pred, ))
print ('Precision score on test set: %.3f' % precision_score(y_test, y_pred))
print ('Recall score on test set: %.3f' % recall_score(y_test, y_pred))
print ("Classification Report on test set:\n", classification_report (y_test, y_pred, digits = 3))
print ("Confusion matrix of test set:\n", confusion_matrix (y_test, y_pred))

Accuracy on test set: 0.9625468164794008
Correct predictions on test set: 1799
Incorrect predictions on test set: 70
F1 score on test set: 0.9598163030998851
Precision score on test set: 0.971
Recall score on test set: 0.949
Classification Report on test set:
               precision    recall  f1-score   support

           0      0.955     0.975     0.965       988
           1      0.971     0.949     0.960       881

    accuracy                          0.963      1869
   macro avg      0.963     0.962     0.962      1869
weighted avg      0.963     0.963     0.963      1869

Confusion matrix of test set:
 [[963  25]
 [ 45 836]]


In [ ]:
df.sample(1)

,sentiment,content
4680,0,not recommended at all very bad worst quality ...


**Taking user input and making predictions from trained model**

In [ ]:
classifier = pipeline("text-classification", model='/content/test-trainer/checkpoint-3500/', return_all_scores=False)

user_input = input("Enter the text you want to predict: ")

prediction = classifier(user_input)

label = prediction[0]['label']

if label == "LABEL_0":
  print('The review is negative.')
elif label == "LABEL_1":
  print('The review is positive.')

Enter the text you want to predict: very poor it is very bad skating board wast money tair is bad
The review is negative.
